<h2
    style="font-size:20px; font-family:'Helvetica Neue'" align="right"> Viktor Trokhymenko 
</h2>
<h3 
    style="font-size:16px; font-family:'Helvetica Neue'" align="right"> @vtrokhymenko
</h3>

In [181]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
np.random.seed(42)
import re
from pprint import pprint
import ast

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [182]:
from pymystem3 import Mystem
m=Mystem()
m.lemmatize('машиной')[0]

'машина'

In [183]:
stopword_ru=stopwords.words('russian')
stopword_ru.append('м')
stopword_ru.append('это')
stopword_ru.append('по')
stopword_ru.append('-')
len(stopword_ru)

155

* ## lenta

In [6]:
df = pd.read_csv('data/news_lenta.csv')
print(df.shape)
df.head()

(699777, 5)


,tags,text,title,topic,url
0,Бокс и ММА,"Украинский боксер Александр Усик рассказал, чт...",Усик поделился планами на будущее после победы...,Спорт,https://lenta.ru/news/2018/07/22/usik_plany/
1,Политика,Израиль эвакуировал из Сирии активистов «Белых...,Израиль вывез из Сирии членов «Белых касок» по...,Мир,https://lenta.ru/news/2018/07/22/evacuation/
2,Футбол,Лондонский «Челси» отказался продать мадридско...,«Челси» отказался продать футболиста «Реалу» з...,Спорт,https://lenta.ru/news/2018/07/22/hazard_real/
3,Бокс и ММА,Вице-президент Федерации профессионального бок...,Слова Порошенко о победе Усика сочли идиотизмом,Спорт,https://lenta.ru/news/2018/07/22/otvet/
4,Музыка,Журнал Billboard опубликовал рейтинг самых выс...,Названы самые богатые музыканты,Культура,https://lenta.ru/news/2018/07/22/moneyrich/


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699777 entries, 0 to 699776
Data columns (total 5 columns):
tags     675534 non-null object
text     699777 non-null object
title    699777 non-null object
topic    699580 non-null object
url      699777 non-null object
dtypes: object(5)
memory usage: 26.7+ MB


### препроцессинг

выделяем с урлы дату публикации & отдельно делаем год и месяц

In [7]:
%%time
df['date_raw'] = df['url'].apply(re.compile('(\d{4})/(\d{2})/(\d{2})').findall)

df['year'] = df['date_raw'].apply(lambda x: int(x[0][0]))
df['month'] = df['date_raw'].apply(lambda x: int(x[0][1]))

<input>:1: DeprecationWarning: invalid escape sequence \d
<input>:1: DeprecationWarning: invalid escape sequence \d
<input>:1: DeprecationWarning: invalid escape sequence \d
<unknown>:1: DeprecationWarning: invalid escape sequence \d


CPU times: user 2.01 s, sys: 125 ms, total: 2.13 s
Wall time: 2.14 s


In [8]:
df.head()

,tags,text,title,topic,url,date_raw,year,month
0,Бокс и ММА,"Украинский боксер Александр Усик рассказал, чт...",Усик поделился планами на будущее после победы...,Спорт,https://lenta.ru/news/2018/07/22/usik_plany/,"[(2018, 07, 22)]",2018,7
1,Политика,Израиль эвакуировал из Сирии активистов «Белых...,Израиль вывез из Сирии членов «Белых касок» по...,Мир,https://lenta.ru/news/2018/07/22/evacuation/,"[(2018, 07, 22)]",2018,7
2,Футбол,Лондонский «Челси» отказался продать мадридско...,«Челси» отказался продать футболиста «Реалу» з...,Спорт,https://lenta.ru/news/2018/07/22/hazard_real/,"[(2018, 07, 22)]",2018,7
3,Бокс и ММА,Вице-президент Федерации профессионального бок...,Слова Порошенко о победе Усика сочли идиотизмом,Спорт,https://lenta.ru/news/2018/07/22/otvet/,"[(2018, 07, 22)]",2018,7
4,Музыка,Журнал Billboard опубликовал рейтинг самых выс...,Названы самые богатые музыканты,Культура,https://lenta.ru/news/2018/07/22/moneyrich/,"[(2018, 07, 22)]",2018,7


In [9]:
df.isnull().sum()

tags        24243
text            0
title           0
topic         197
url             0
date_raw        0
year            0
month           0
dtype: int64

In [10]:
def clean_text(text):
    
    #убираем всякий мусор
    text = re.sub('-\s\r\n\s+|-\s\r\n|\r\n|[«»]|[><]', ' ', text.lower())    
    text = re.sub('[.,:;_%©?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|"|–|—|', ' ', text)
    text = re.sub('\’|\”|‘', ' ', text)
    text = re.sub(r'[\\n]|[\d]', '', text)
    text = re.sub(r'ё', 'е', text)
    
    return text

clean_text('куала-лумпур 123 маш123и&а')

<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \s
<ipython-input-10-a4a721847d48>:4: DeprecationWarning: invalid escape sequence \s
  text = re.sub('-\s\r\n\s+|-\s\r\n|\r\n|[«»]|[><]', ' ', text.lower())


'куала-лумпур  маши а'

In [11]:
%%time
df['text_clean'] = df['text'].apply(clean_text)

CPU times: user 1min 49s, sys: 3.01 s, total: 1min 52s
Wall time: 1min 54s


In [12]:
df.loc[np.random.randint(low=0, high=df.shape[0])].text_clean#.strip()

'телеканал   подмосковье  опубликовал видео со свалки из трупов собак в пушкинском районе подмосковья  трупы животных нашли местные жители  которые рассказали об этом зоозащитникам  первоначально сообщалось  что свалка находится на окраине ивантеевки  однако администрация города уточнила  что трупы найдены у поселка левково  в гу мвд по области агентству тасс сообщили  что по факту обнаружения мертвых животных начата проверка '

In [280]:
#убираем стоп-слова и лемматизируем с помощью mystem
#
#!!! для mystem это не правильно !!!
#
def stop_words_lemma(text):
    #print('\n',text)
    words = word_tokenize(text)#text.split()
    #print('\n',words)

    #внизу костыль, который был написан перед тем как ложиться спать, тк m.lemmatize('-центробанка')->'-'
    words_lem=[]
    for w in words:
        #print(w)
        if w[0]=='-':
            try:
                if not w[1:] in stopword_ru:
                    words_lem.append(m.lemmatize(w[1:])[0])
            except: pass
        else:
            if not w in stopword_ru:
                words_lem.append(m.lemmatize(w)[0])
    
    #words = [m.lemmatize(w)[0] for w in words if not w in stopword_ru]
    
    
    return words_lem

stop_words_lemma('по заказу- -центробанка  торгово-промышленной')
#stop_words_lemma('предлагают за - миллионов')

['заказ', 'центробанк', 'торгово-промышленный']

In [281]:
%%time
df['text_clear_lemma'] = df['text_clean'].apply(stop_words_lemma)

CPU times: user 1h 9min 5s, sys: 18min 51s, total: 1h 27min 56s
Wall time: 3h 9min 7s


In [ ]:
df.head()

In [283]:
df.to_csv('data/news_lenta_clean.csv',index=False)

In [13]:
#лемматизируем сразу весь текст с помощью mystem, а потом убираем стоп-слова

def lemma_stop_words(text):
    
    #text_new=clean_text(''.join(m.lemmatize(text)))
    text_new=''.join(m.lemmatize(text))
    
    words_new=[]
    for w in word_tokenize(text_new):
        #print(w)
        if w[0]=='-':
            try:
                if not w[1:] in stopword_ru:
                    words_new.append(w[1:])
            except:pass
        elif w[-1]=='-':
            try:
                if not w[:-1] in stopword_ru:
                    words_new.append(w[:-1])
            except:pass
        else:
            if not w in stopword_ru:
                words_new.append(w)
    
    return words_new

lemma_stop_words('по заказу- -центробанка  торгово-промышленной')
#lemma_stop_words('комитетов палаты лордов — верхней палаты')

['заказ', 'центробанк', 'торгово-промышленный']

In [14]:
%%time
df['text_clean_lemma1'] = df['text_clean'].apply(lemma_stop_words)

CPU times: user 21min 3s, sys: 1min 14s, total: 22min 18s
Wall time: 1h 58min 9s


In [19]:
df.head(2)

,tags,text,title,topic,url,date_raw,year,month,text_clean,text_clean_lemma1
0,Бокс и ММА,"Украинский боксер Александр Усик рассказал, чт...",Усик поделился планами на будущее после победы...,Спорт,https://lenta.ru/news/2018/07/22/usik_plany/,"[(2018, 07, 22)]",2018,7,украинский боксер александр усик рассказал чт...,"[украинский, боксер, александр, усик, рассказы..."
1,Политика,Израиль эвакуировал из Сирии активистов «Белых...,Израиль вывез из Сирии членов «Белых касок» по...,Мир,https://lenta.ru/news/2018/07/22/evacuation/,"[(2018, 07, 22)]",2018,7,израиль эвакуировал из сирии активистов белых...,"[израиль, эвакуировать, сирия, активист, белый..."


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699777 entries, 0 to 699776
Data columns (total 10 columns):
tags                 699777 non-null object
text                 699777 non-null object
title                699777 non-null object
topic                699777 non-null object
url                  699777 non-null object
date_raw             699777 non-null object
year                 699777 non-null int64
month                699777 non-null int64
text_clean           699777 non-null object
text_clean_lemma1    699777 non-null object
dtypes: int64(2), object(8)
memory usage: 53.4+ MB


In [55]:
df.fillna(0,inplace=True)
df.isnull().sum()

tags                 0
text                 0
title                0
topic                0
url                  0
date_raw             0
year                 0
month                0
text_clean           0
text_clean_lemma1    0
dtype: int64

In [76]:
df=df[df.len>2]
df.shape

(699513, 12)

In [166]:
%%time
#df.to_csv('data/news_lenta_mysterm1.csv',index=False,sep=';',encoding='utf-8')
#df[['tags','topic','year','month','text_clean','text_clean_lemma1']].to_csv('data/news_lenta_mysterm1.csv',index=False)
df[['tags','topic','year','month','text_clean']].to_csv('data/news_lenta_mysterm1.csv',index=False)

CPU times: user 26.5 s, sys: 14.4 s, total: 40.8 s
Wall time: 57.3 s


In [117]:
df.shape

(699512, 12)

In [162]:
df.loc[405320:405420]

,tags,text,title,topic,url,date_raw,year,month,text_clean,text_clean_lemma1,699777,len
405320,Все,Житель деревни в британском округе Дувр выраст...,В саду британского пенсионера выросло яблоко-м...,Из жизни,https://lenta.ru/news/2009/09/25/apple/,"[(2009, 09, 25)]",2009,9,житель деревни в британском округе дувр выраст...,"[житель, деревня, британский, округ, дувр, выр...",121,121
405321,Все,В Челябинской области завершено следствие по д...,Челябинского подростка обвинили в ограблении ю...,Россия,https://lenta.ru/news/2009/09/25/troick/,"[(2009, 09, 25)]",2009,9,в челябинской области завершено следствие по д...,"[челябинский, область, завершать, следствие, д...",101,101
405322,Все,"Главный тренер лондонского футбольного клуба ""...","Тренер ""Челси"" отложил дебют Жиркова в чемпион...",Спорт,https://lenta.ru/news/2009/09/25/coach/,"[(2009, 09, 25)]",2009,9,главный тренер лондонского футбольного клуба ...,"[главный, тренер, лондонский, футбольный, клуб...",101,101
405323,Все,Крупнейший российский производитель алюминия к...,"""Русал"" решил погасить задолженность перед Аль...",Экономика,https://lenta.ru/news/2009/09/25/rusal1/,"[(2009, 09, 25)]",2009,9,крупнейший российский производитель алюминия к...,"[крупный, российский, производитель, алюминий,...",146,146
405324,Все,Компания Walt Disney Studios отказалась от съе...,"Новая версия ""Дневника Анны Франк"" напугала ру...",Культура,https://lenta.ru/news/2009/09/25/frank/,"[(2009, 09, 25)]",2009,9,компания walt disey studios отказалась от съем...,"[компания, walt, disey, studios, отказываться,...",108,108
405325,Все,Офис генерального директора крупнейшего в Евро...,Офис главы крупнейшего банка Европы переедет в...,Экономика,https://lenta.ru/news/2009/09/25/hsbc/,"[(2009, 09, 25)]",2009,9,офис генерального директора крупнейшего в евро...,"[офис, генеральный, директор, крупный, европа,...",108,108
405326,Все,"Президент итальянского футбольного клуба ""Напо...",Бывшему тренеру сборной Италии по футболу пост...,Спорт,https://lenta.ru/news/2009/09/25/donadoni/,"[(2009, 09, 25)]",2009,9,президент итальянского футбольного клуба напо...,"[президент, итальянский, футбольный, клуб, нап...",133,133
405327,Все,"На оперативно-стратегических учениях ""Запад-20...",Российские ВВС опробуют в Белоруссии новую сис...,Наука и техника,https://lenta.ru/news/2009/09/25/targeting/,"[(2009, 09, 25)]",2009,9,на оперативно-стратегических учениях запад- ...,"[оперативно-стратегический, учение, запад, бом...",138,138
405328,Все,Гражданин США обвинен в преступном сговоре с ц...,Житель Бруклина обвинен в подготовке убийств с...,Россия,https://lenta.ru/news/2009/09/25/murder/,"[(2009, 09, 25)]",2009,9,гражданин сша обвинен в преступном сговоре с ц...,"[гражданин, сша, обвинять, преступный, сговор,...",178,178
405329,Все,"В городе Питтсбург, штат Пенсильвания, в четве...",В Питтсбурге разогнали акцию протеста против с...,Мир,https://lenta.ru/news/2009/09/25/crowd/,"[(2009, 09, 25)]",2009,9,в городе питтсбург штат пенсильвания в четве...,"[город, питтсбург, штат, пенсильвания, четверг...",158,158


In [168]:
#df_test=pd.read_csv('data/news_lenta_mysterm1.csv', sep=';',encoding='utf-8')
df_test=pd.read_csv('data/news_lenta_mysterm1.csv') 
print(df_test.shape)
df_test.tail()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb0 in position 0: invalid start byte

In [161]:
df_test.loc[405320:405420]

,tags,topic,year,month,text_clean,text_clean_lemma1
405320,Все,Россия,2009.0,9.0,в челябинской области завершено следствие по д...,"['челябинский', 'область', 'завершать', 'следс..."
405321,Все,Спорт,2009.0,9.0,главный тренер лондонского футбольного клуба ...,"['главный', 'тренер', 'лондонский', 'футбольны..."
405322,Все,Экономика,2009.0,9.0,крупнейший российский производитель алюминия к...,"['крупный', 'российский', 'производитель', 'ал..."
405323,Все,Культура,2009.0,9.0,компания walt disey studios отказалась от съем...,"['компания', 'walt', 'disey', 'studios', 'отка..."
405324,Все,Экономика,2009.0,9.0,офис генерального директора крупнейшего в евро...,"['офис', 'генеральный', 'директор', 'крупный',..."
405325,Все,Спорт,2009.0,9.0,президент итальянского футбольного клуба напо...,"['президент', 'итальянский', 'футбольный', 'кл..."
405326,Все,Наука и техника,2009.0,9.0,на оперативно-стратегических учениях запад- ...,"['оперативно-стратегический', 'учение', 'запад..."
405327,Все,Россия,2009.0,9.0,гражданин сша обвинен в преступном сговоре с ц...,"['гражданин', 'сша', 'обвинять', 'преступный',..."
405328,Все,Мир,2009.0,9.0,в городе питтсбург штат пенсильвания в четве...,"['город', 'питтсбург', 'штат', 'пенсильвания',..."
405329,Все,Спорт,2009.0,9.0,главный тренер сборной бразилии по футболу дун...,"['главный', 'тренер', 'сборная', 'бразилия', '..."


In [120]:
df_test.head()

,tags,topic,year,month,text_clean,text_clean_lemma1
0,Бокс и ММА,Спорт,2018.0,7.0,украинский боксер александр усик рассказал чт...,"['украинский', 'боксер', 'александр', 'усик', ..."
1,Политика,Мир,2018.0,7.0,израиль эвакуировал из сирии активистов белых...,"['израиль', 'эвакуировать', 'сирия', 'активист..."
2,Футбол,Спорт,2018.0,7.0,лондонский челси отказался продать мадридско...,"['лондонский', 'челси', 'отказываться', 'прода..."
3,Бокс и ММА,Спорт,2018.0,7.0,вице-президент федерации профессионального бок...,"['вице-президент', 'федерация', 'профессиональ..."
4,Музыка,Культура,2018.0,7.0,журнал billboard опубликовал рейтинг самых выс...,"['журнал', 'billboard', 'опубликовывать', 'рей..."


In [169]:
df.head(2)

,tags,text,title,topic,url,date_raw,year,month,text_clean,text_clean_lemma1,699777,len
0,Бокс и ММА,"Украинский боксер Александр Усик рассказал, чт...",Усик поделился планами на будущее после победы...,Спорт,https://lenta.ru/news/2018/07/22/usik_plany/,"[(2018, 07, 22)]",2018,7,украинский боксер александр усик рассказал чт...,"[украинский, боксер, александр, усик, рассказы...",112,112
1,Политика,Израиль эвакуировал из Сирии активистов «Белых...,Израиль вывез из Сирии членов «Белых касок» по...,Мир,https://lenta.ru/news/2018/07/22/evacuation/,"[(2018, 07, 22)]",2018,7,израиль эвакуировал из сирии активистов белых...,"[израиль, эвакуировать, сирия, активист, белый...",87,87


In [ ]:
df['range']=range(data.shape[0])

In [177]:
df[['text','year','month','text_clean_lemma1']].to_csv('data/news_lenta_mysterm1.csv',index=False)

In [179]:
df_test=pd.read_csv('data/news_lenta_mysterm1.csv') 
print(df_test.shape)
df_test.head()

(699512, 4)


,text,year,month,text_clean_lemma1
0,"Украинский боксер Александр Усик рассказал, чт...",2018,7,"['украинский', 'боксер', 'александр', 'усик', ..."
1,Израиль эвакуировал из Сирии активистов «Белых...,2018,7,"['израиль', 'эвакуировать', 'сирия', 'активист..."
2,Лондонский «Челси» отказался продать мадридско...,2018,7,"['лондонский', 'челси', 'отказываться', 'прода..."
3,Вице-президент Федерации профессионального бок...,2018,7,"['вице-президент', 'федерация', 'профессиональ..."
4,Журнал Billboard опубликовал рейтинг самых выс...,2018,7,"['журнал', 'billboard', 'опубликовывать', 'рей..."


---

* ## novayagazeta 

In [180]:
#https://github.com/buriy/russian-nlp-datasets/releases
df = pd.read_csv('data/novayagazeta.ru.txt', sep='\t', encoding='utf-8', header=None)
df.columns = ['id', 'url', 'title', 'text']

print('before filter duplicates: ',df.shape)
df = df.drop_duplicates('url')
print('after filter duplicates: ',df.shape)
df.head()

before filter duplicates:  (14491, 4)
after filter duplicates:  (14485, 4)


,id,url,title,text
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са..."
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России..."


In [184]:
def clean_text(text):
    
    #убираем всякий мусор
    text = re.sub('-\s\r\n\s+|-\s\r\n|\r\n|[«»]|[><]', ' ', text.lower())    
    text = re.sub('[.,:;_%©?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|"|–|—|', ' ', text)
    text = re.sub('\’|\”|‘', ' ', text)
    text = re.sub(r'[\\n]|[\d]', '', text)
    text = re.sub(r'ё', 'е', text)
    
    return text

clean_text('куала-лумпур 123 маш123и&а')

'куала-лумпур  маши а'

In [185]:
%%time
df['text_clean'] = df['text'].apply(clean_text)

CPU times: user 10.9 s, sys: 32.9 s, total: 43.8 s
Wall time: 1min 15s


In [12]:
df.loc[np.random.randint(low=0, high=df.shape[0])].text_clean#.strip()

'телеканал   подмосковье  опубликовал видео со свалки из трупов собак в пушкинском районе подмосковья  трупы животных нашли местные жители  которые рассказали об этом зоозащитникам  первоначально сообщалось  что свалка находится на окраине ивантеевки  однако администрация города уточнила  что трупы найдены у поселка левково  в гу мвд по области агентству тасс сообщили  что по факту обнаружения мертвых животных начата проверка '

In [186]:
#лемматизируем сразу весь текст с помощью mystem, а потом убираем стоп-слова

def lemma_stop_words(text):
    
    #text_new=clean_text(''.join(m.lemmatize(text)))
    text_new=''.join(m.lemmatize(text))
    
    words_new=[]
    for w in word_tokenize(text_new):
        #print(w)
        if w[0]=='-':
            try:
                if not w[1:] in stopword_ru:
                    words_new.append(w[1:])
            except:pass
        elif w[-1]=='-':
            try:
                if not w[:-1] in stopword_ru:
                    words_new.append(w[:-1])
            except:pass
        else:
            if not w in stopword_ru:
                words_new.append(w)
    
    return words_new

lemma_stop_words('по заказу- -центробанка  торгово-промышленной')
#lemma_stop_words('комитетов палаты лордов — верхней палаты')

['заказ', 'центробанк', 'торгово-промышленный']

In [187]:
%%time
df['text_clean_lemma'] = df['text_clean'].apply(lemma_stop_words)

CPU times: user 2min 5s, sys: 1min 38s, total: 3min 44s
Wall time: 9min 22s


In [188]:
df.head()

,id,url,title,text,text_clean,text_clean_lemma
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",как и предсказывал на страницах а также на са...,"[предсказывать, страница, также, сайт, новый, ..."
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...,август освобожденные заключенные на площади ...,"[август, освобождать, заключать, площадь, опер..."
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...,август в россии время трагических годовщин ...,"[август, россия, время, трагический, годовщина..."
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...,в -м избирательном округе районы выхино-жулеб...,"[избирательный, округ, район, выхино-жулебино,..."
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России...",белоруссия и казахстан с которыми у россии ...,"[белоруссия, казахстан, который, россия, тамож..."


In [191]:
%%time
df[['id','text_clean_lemma']].to_csv('data/novayagazeta_mystem1.csv',index=False)

CPU times: user 1.49 s, sys: 116 ms, total: 1.6 s
Wall time: 1.64 s


In [193]:
df_test=pd.read_csv('data/novayagazeta_mystem1.csv') 
print(df_test.shape)
df_test.head()

(14485, 2)


,id,text_clean_lemma
0,1408695844,"['предсказывать', 'страница', 'также', 'сайт',..."
1,1408695844,"['август', 'освобождать', 'заключать', 'площад..."
2,1408695844,"['август', 'россия', 'время', 'трагический', '..."
3,1408695844,"['избирательный', 'округ', 'район', 'выхино-жу..."
4,1408695844,"['белоруссия', 'казахстан', 'который', 'россия..."
